# Reconocimiento de dígitos con KNN

Veamos como interactuar con las clases implementadas en el módulo de C++ haciendo una experimentación muy simplificada, y a modo de ejemplo, del método KNN para reconocer digitos

## Compilando el módulo desde C++

Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [3]:
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 9.3.0
-- The CXX compiler identification is GNU 9.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /home/sebasbocaccio/anaconda3/envs/tp2/bin/python (found version "3.6.5") 
-- Found PythonLibs: /home/sebasbocaccio/anaconda3/envs/tp2/lib/libpython3.6m.so
-- pybind11 v2.2.4
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home/sebasbocaccio/Desktop/FACU/Metodos/

make[2]: *** [CMakeFiles/metnum.dir/build.make:63: CMakeFiles/metnum.dir/src/metnum.cpp.o] Error 1
make[1]: *** [CMakeFiles/Makefile2:125: CMakeFiles/metnum.dir/all] Error 2
make: *** [Makefile:130: all] Error 2


In [1]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import metnum
import numpy

/home/sebasbocaccio/Desktop/FACU/Metodos/tp2-metodos/tp2/notebooks
Python 3.6.5 :: Anaconda, Inc.


## Explorando los datos

In [39]:
%load_ext autoreload
%autoreload 2
import pandas as pd

df_train = pd.read_csv("../data/train.csv")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Es un csv, con columnas por cada pixel

### Reducir el dataset

Ya que el dataset es grande, me quedo con una porción de éste

**<font color=red>ESTO ES SOLO PARA HACER PRUEBAS. CUANDO REPORTEN LOS RESULTADOS, USAR DATASET COMPLETO</font>**

In [40]:
df_train = df_train[:1000]

En el primer caracter está el dígito a reconocer. Llamamos a esto `y_train`

In [41]:
# Uso values para mandar todo a arrays de numpy
X = df_train[df_train.columns[1:]].values
y = df_train["label"].values.reshape(-1, 1)

X.shape, y.shape

((1000, 784), (1000, 1))

Convierto el resto a matrices de 28 x 28

A ver cómo se ven...

In [42]:
import matplotlib.pyplot as plt

num = 1917

print(f"Supuestamente es un {int(y[num])}")

img = X[num].reshape(28, 28)
plt.imshow(img, cmap="Greys")


IndexError: index 1917 is out of bounds for axis 0 with size 1000

### Balance de clases

¿Tenemos la misma cantidad de cada número?

In [43]:
df_train["label"].value_counts(normalize=True)

2    0.124
0    0.107
7    0.105
4    0.102
9    0.097
6    0.097
1    0.096
8    0.093
3    0.090
5    0.089
Name: label, dtype: float64

No: tenemos más 1 que 5, pero tampoco está tan desbalanceado

### Conjunto de development

In [48]:
limit = int(0.8 * X.shape[0]) 

X_train, y_train = X[:limit], y[:limit]
dist = []
for i in range(0,len(X_train),1):
    dist.append((numpy.inner(X_train[i], X_train[i]),y_train[i][0]))
dist.sort()
for i in dist:
    print(i[1])
X_val, y_val = X[limit:], y[limit:]

assert len(X_train) == len(y_train)
assert len(X_val) == len(y_val)

print(f"Ahora tengo {len(X_train)} instancias de entrenamiento y {len(X_val)} de validación")


1
1
1
7
1
1
5
1
1
1
1
1
1
2
6
1
1
1
6
1
1
7
9
1
1
3
9
1
1
1
1
1
1
1
1
1
1
4
4
4
1
1
2
1
1
9
1
1
7
1
1
5
1
1
1
9
9
1
5
4
9
9
7
1
6
1
4
1
4
1
3
1
1
9
8
2
1
7
5
1
0
8
1
1
9
7
1
5
5
1
1
7
6
3
6
1
7
5
6
9
7
7
9
8
9
7
7
5
6
2
3
8
1
4
5
8
5
3
5
5
6
5
9
5
7
2
9
8
3
3
1
2
5
2
8
3
9
7
5
5
1
2
7
0
4
7
8
9
8
5
0
3
9
3
9
1
9
7
9
5
7
3
1
5
9
7
3
7
1
9
7
5
1
9
4
4
2
7
1
3
1
7
9
7
7
2
2
9
4
3
6
2
7
1
4
2
5
3
2
4
8
9
0
6
3
4
1
6
5
4
1
2
8
9
7
4
0
3
6
0
8
3
9
3
9
1
5
2
2
7
9
4
2
7
7
7
6
2
4
7
4
7
6
5
1
1
9
3
9
5
2
6
1
4
6
1
3
7
5
3
5
3
3
8
1
0
4
4
9
8
0
1
1
7
2
4
0
8
8
1
3
9
5
3
0
2
6
2
6
2
4
3
2
5
9
6
2
5
7
4
1
4
3
2
4
3
0
3
5
1
9
2
9
7
5
5
9
2
8
5
7
9
2
9
9
3
2
9
4
7
0
9
4
9
4
4
8
7
6
2
1
4
5
7
6
4
3
7
2
6
5
7
9
5
9
3
7
5
4
5
6
8
4
4
7
2
3
6
8
7
8
6
8
2
4
6
4
7
9
6
4
4
7
8
6
4
7
5
6
2
8
4
7
8
1
9
0
7
4
4
2
7
5
3
9
7
0
0
9
7
8
9
1
3
9
6
3
8
7
0
0
4
9
1
9
5
2
5
9
6
8
6
4
9
2
4
3
8
2
2
9
0
8
0
2
0
6
6
5
3
3
0
5
4
6
6
0
9
4
0
3
3
0
3
6
5
0
8
5
6
4
7
5
5
6
3
5
4
8
5
4
4
6
0
6
2
3
2
7
7
6
4
0
2
7
0
7
9
5
5


In [45]:
from sklearn.neighbors import KNeighborsClassifier

# Acá pueden cambiar el clasificador nuestro por el de sklearn!

clf = metnum.KNNClassifier(80)
#clf = KNeighborsClassifier(1)
clf.fit(X_train, y_train)


In [46]:
%%time
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_val)
print(y_pred)
dist = []

for i in range(0,len(X_val),1):
    dist.append((numpy.inner(X_val[i], X_val[i])))
print(dist)
acc = accuracy_score(y_val, y_pred)
print("Accuracy: {}".format(acc))


[4. 0. 1. 1. 0. 2. 0. 1. 0. 2. 5. 9. 0. 0. 7. 2. 4. 2. 4. 2. 9. 9. 7. 0.
 2. 0. 1. 1. 0. 1. 5. 0. 9. 6. 4. 0. 7. 0. 1. 0. 2. 1. 0. 0. 2. 7. 0. 4.
 2. 7. 0. 4. 0. 5. 0. 7. 9. 7. 7. 0. 4. 1. 0. 4. 0. 4. 1. 4. 1. 1. 2. 2.
 4. 2. 1. 1. 4. 0. 2. 1. 2. 4. 9. 2. 7. 2. 0. 2. 0. 0. 0. 4. 2. 1. 0. 1.
 2. 0. 2. 3. 2. 5. 6. 0. 2. 0. 0. 1. 2. 7. 7. 5. 0. 2. 9. 2. 1. 7. 0. 0.
 0. 0. 0. 4. 4. 1. 0. 2. 0. 2. 6. 7. 1. 0. 2. 9. 2. 7. 5. 4. 2. 2. 2. 2.
 4. 1. 2. 1. 0. 5. 2. 4. 4. 4. 0. 7. 2. 2. 0. 2. 0. 9. 0. 7. 5. 4. 0. 4.
 0. 1. 4. 1. 2. 0. 9. 3. 0. 0. 1. 2. 2. 0. 0. 1. 5. 4. 5. 2. 1. 1. 2. 0.
 2. 0. 2. 9. 5. 7. 4. 1.]
[5244743, 6099963, 4571489, 2356634, 6937874, 9656591, 5746575, 2867721, 5747873, 8535980, 3623772, 4903751, 7491237, 6821361, 4026291, 8397335, 5621274, 8346879, 5646005, 7958260, 4180610, 5691961, 6499467, 5890395, 4391251, 7154350, 2989211, 2721131, 7262950, 3398324, 3650938, 6662520, 4906912, 5985598, 5298592, 10133406, 5438826, 11078816, 3333344, 7067042, 4870868, 4570635, 7116347, 

# Testeo

In [50]:
df_train = pd.read_csv("../data/custom_test.csv")



# Uso values para mandar todo a arrays de numpy
X = df_train[df_train.columns[1:]].values
y = df_train["label"].values.reshape(-1, 1)

X.shape, y.shape




((6, 2), (6, 1))

In [51]:
x_train = X[:3]
y_train = y[:3]
x_test = X[3:]
y_test = y[3:]

clf = metnum.KNNClassifier(2)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
y_pred

array([1., 2., 1.])

# Testeo 2 